# HW06: Transformers and Doc Embeddings

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [1]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

--2023-04-05 17:53:36--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M  --.-KB/s    in 0.1s    

2023-04-05 17:53:38 (291 MB/s) - ‘train.csv’ saved [29470338/29470338]



,label,title,lead,text
69209,sci/tech,Google v. Microsoft: Trustbusters Not Needed,"Last week, Google, a company renowned for its ...",Google v. Microsoft: Trustbusters Not Needed L...
24653,world,War on terror to continue in Waziristan: Pak.,"Islamabad, Sept 12. (UNI): As hostilities cont...",War on terror to continue in Waziristan: Pak. ...
90911,sport,Given punished as Newcastle fall to fourth str...,Newcastle United goalkeeper Shay Given was adj...,Given punished as Newcastle fall to fourth str...
97459,sport,Colonials Come on Late,"Trailing at halftime to an overmatched foe, Ge...",Colonials Come on Late Trailing at halftime to...
3484,business,Nortel announces Sprint extension worth US\$1B...,TORONTO (CP) - Sprint Communications Co. has a...,Nortel announces Sprint extension worth US\$1B...


## Hugginface Transformers

In [3]:
# !pip install transformers
from transformers import DistilBertForSequenceClassification, DistilBertConfig, DistilBertTokenizerFast
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.2 MB/s eta 0:00:00


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##TODO build a transformer model to do sequence classification with the goal to predict the label from the text

model_name = 'distilbert-base-uncased' # huggingface model_ID or path to folder 
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

In [11]:
##TODO print the summary of the model

print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [16]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np

##TODO split the sample into a training and a test set 

label2id = {"world": 0, "sport": 1, "business": 2, "sci/tech": 3}
y = df["label"].apply(lambda x: label2id[x]).values

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), y.tolist(), test_size = .2)


##TODO prepare the dataset for torch.

class GenericDataset(Dataset):
    def __init__(self, X, y, tokenizer):
        self.X = X
        self.y = y
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        text = self.X[index]
        tokens = self.tokenizer(text, padding=True, return_tensors='pt', trancation=True, max_length=256)
        return tokens['input_ids'][0], self.y[index]

train_dataset = GenericDataset(X_train, y_train, tokenizer)
test_dataset = GenericDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False, drop_last=True)

In [17]:
##TODO fit the model and print the obtained accuracy (hint: you can follow the training steps in the notebook. To learn more, checkout the trainer class of huggingface transformers)

from tqdm import tqdm
from sklearn.metrics import accuracy_score

model.to(device)
num_epochs = 5

optimizer = torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):
    model.train()
    for text, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs, labels=labels)
        loss, logits = outputs[:2]
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    model.eval()
    predictions = []
    true_labels = []
    for text, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
            logits = outputs[0]
            prediction = torch.argmax(logits, dim=1).detach().cpu().numpy()
            predictions.append(prediction)
            true_labels.append(labels.cpu().numpy())


    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Epoch {epoch+1}, Test accuracy: {accuracy:.4f}")

RuntimeError: ignored

# Doc Embedding

In [ ]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip 

In [ ]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

In [ ]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))


In [ ]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

In [ ]:
##TODO train Doc2Vec on the texts in the dataset
##TODO derive the word vectors for each text in the dataset
##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)

In [ ]:
##TODO do the same with embeddings provided by spaCy

In [ ]:
##TODO do the same with SBERT embeddings